In [1]:
import numpy as np

In [2]:
def EqualityQPSolverLUdense(H, g, A, b):
    n = H.shape[0]
    m = A.shape[0]
    KKT = np.block([
        [H, -A.T],
        [A, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-g, b])
    lu, piv = lu_factor(KKT)
    sol = lu_solve((lu, piv), rhs)
    x = sol[:n]
    lam = sol[n:]
    return x, lam

In [3]:
def EqualityQPSolverLUsparse(H, g, A, b):
    n = H.shape[0]
    m = A.shape[0]
    Hs = csc_matrix(H)
    As = csc_matrix(A)
    KKT = csc_matrix(np.block([
        [Hs.toarray(), -As.T.toarray()],
        [As.toarray(), np.zeros((m, m))]
    ]))
    rhs = np.concatenate([-g, b])
    lu = splu(KKT)
    sol = lu.solve(rhs)
    x = sol[:n]
    lam = sol[n:]
    return x, lam

In [5]:
def EqualityQPSolverLDLdense(H, g, A, b):
    n = H.shape[0]
    m = A.shape[0]
    KKT = np.block([
        [H, -A.T],
        [A, np.zeros((m, m))]
    ])
    rhs = np.concatenate([-g, b])
    L, D, perm = ldl(KKT)
    y = np.linalg.solve(L, rhs[perm])
    z = np.linalg.solve(D, y)
    sol = np.linalg.solve(L.T, z)
    x = sol[:n]
    lam = sol[n:]
    return x, lam

In [6]:

from scipy.linalg import lu_factor, lu_solve, ldl
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu, spsolve
from scipy.sparse import bmat, csc_matrix
from scipy.sparse import csc_matrix, bmat
import qdldl
from qdldl import spldlt_solve


def EqualityQPSolverLDLsparse(H, g, A, b):
    n = H.shape[0]
    m = A.shape[0]

    # KKT system
    Hs = csc_matrix(H)
    As = csc_matrix(A)
    zero = csc_matrix((m, m))
    KKT = bmat([
        [Hs, -As.T],
        [As, zero]
    ], format="csc")

    rhs = np.concatenate([-g, b])

    # Solve using QDLDL
    sol = spldlt_solve(KKT, rhs)

    x = sol[:n]
    lam = sol[n:]
    return x, lam


ImportError: cannot import name 'spldlt_solve' from 'qdldl' (/Users/lottealstrup/opt/anaconda3/envs/AI/lib/python3.10/site-packages/qdldl.cpython-310-darwin.so)

In [7]:
def EqualityQPSolverRangeSpace(H, g, A, b):
    # Range space method
    At = A.T
    S = A @ np.linalg.solve(H, At)
    rhs = b + A @ np.linalg.solve(H, g)
    lam = np.linalg.solve(S, rhs)
    x = -np.linalg.solve(H, g - At @ lam)
    return x, lam

In [8]:
def EqualityQPSolverNullSpace(H, g, A, b):
    # Null space method
    from scipy.linalg import null_space
    Z = null_space(A)
    x0 = np.linalg.lstsq(A, b, rcond=None)[0]
    Hz = Z.T @ H @ Z
    gz = Z.T @ (H @ x0 + g)
    p = -np.linalg.solve(Hz, gz)
    x = x0 + Z @ p
    lam = np.linalg.lstsq(A.T, H @ x + g, rcond=None)[0]
    return x, lam

In [9]:
def EqualityQPSolver(H, g, A, b, solver):
    solvers = {
        "LUdense": EqualityQPSolverLUdense,
        "LUsparse": EqualityQPSolverLUsparse,
        "LDLdense": EqualityQPSolverLDLdense,
        "LDLsparse": EqualityQPSolverLDLsparse,
        "RangeSpace": EqualityQPSolverRangeSpace,
        "NullSpace": EqualityQPSolverNullSpace
    }
    if solver not in solvers:
        raise ValueError(f"Unknown solver: {solver}")
    return solvers[solver](H, g, A, b)

In [10]:
def generate_test_problem(n, beta, alpha):
    # Generate random matrix M with 15% nonzero elements
    M = np.random.randn(n, n)
    mask_M = np.random.rand(n, n) > 0.85
    M[mask_M] = 0
    
    # Generate H = M M' + alpha I
    H = M @ M.T + alpha * np.eye(n)
    
    # Generate random vector g
    g = np.random.randn(n)
    
    # Generate matrix A with 15% nonzero elements
    m = round(beta * n)
    A = np.random.randn(m, n)
    mask_A = np.random.rand(m, n) > 0.85
    A[mask_A] = 0
    
    # Generate random vector b
    b = np.random.randn(m)
    
    return H, g, A, b

def compute_lagrangian(H, g, A, b, x, lam):
    L = 0.5 * x.T @ H @ x + g.T @ x - lam.T @ (A @ x - b)
    return L

In [11]:
# Parameter
n = 100
beta = 0.5  
alpha = 0.1 

# Generate test problem
H, g, A, b = generate_test_problem(n, beta, alpha)

In [ ]:
x, lam, time1, cpu1 = EqualityQPSolver(H, g, A, b, 'LDLdense')
L = compute_lagrangian(H, g, A, b, x, lam)
L, time1, cpu1

NameError: name 'EqualityQPSolverLDLsparse' is not defined